In [1]:
CPU_COUNT = 4 # number of CPUs for parallel generators if not doing parallel experiments, can use CPU_COUNT = multiprocessing.cpu_count()
WORKER_ID = 1
WORKER_COUNT = 8

In [2]:
# whether to log each feature and sequence status
verbose = True

In [10]:
import gc
import os
import pandas as pd
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.append('..')

In [11]:
# setup paths
pwd = os.getcwd().replace("deepvideoclassification","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [12]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()
# make logger aware of any uncaught exceptions
def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    logger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))
sys.excepthook = handle_exception

In [15]:
from deepvideoclassification.architectures import Architecture
from deepvideoclassification.pretrained_CNNs import pretrained_model_names, pretrained_model_names_bucketed

ImportError: cannot import name 'precompute_CNN_features'

# Create experiments lists and run experiments

* batch 1 = run frozen MLPs and LRCNNs and all concats

* batch 2 = run trainable MLP and LRCNN on best performing frozen variants

* batch 3 = run trainable but initializing with best CNN weights

* batch 4 = run C3D models

## Experiment batch 1

In [22]:
experiment_batch_name = 'experiment_batch_1'

In [23]:
# init list of experiments
experiments = []

In [24]:
pooling = 'max'
layer_sizes = [512, 256, 128, 0]
dropouts = [0.2]
sequence_lengths = [1,3,5,10,20]
sequence_models = ["LSTM", "SimpleRNN", "GRU", "Convolution1D"]
sequence_model_layer_counts = [1,2,3]

In [25]:
####################
### image_MLP_frozen 
####################

for pretrained_model_name in pretrained_model_names_bucketed:
    for layer_1_size in layer_sizes:
        for layer_2_size in layer_sizes:
            for layer_3_size in layer_sizes:
                for dropout in dropouts:

                    # build experiment parameters
                    experiment = {}
                    
                    experiment['architecture'] = 'image_MLP_frozen'
                    experiment['sequence_length'] = 1
                    experiment['pretrained_model_name'] = pretrained_model_name
                    experiment['layer_1_size'] = layer_1_size
                    experiment['layer_2_size'] = layer_2_size
                    experiment['layer_3_size'] = layer_3_size
                    experiment['dropout'] = dropout
                    experiment['pooling'] = 'max' # outperforms avg across all parameters
                    
                    # add to list of experiments
                    experiments.append(experiment)

In [26]:
####################
### video_MLP_concat
####################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:

                        # build experiment parameters
                        experiment = {}

                        experiment['architecture'] = 'video_MLP_concat'
                        experiment['pretrained_model_name'] = pretrained_model_name
                        experiment['layer_1_size'] = layer_1_size
                        experiment['layer_2_size'] = layer_2_size
                        experiment['layer_3_size'] = layer_3_size
                        experiment['dropout'] = dropout
                        experiment['pooling'] = 'max' # outperforms avg across all parameters

                        # add to list of experiments
                        experiments.append(experiment)

In [27]:
######################
### video_LRCNN_frozen
######################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:
                        for sequence_model in sequence_models:
                            for sequence_model_layers in sequence_model_layer_counts:

                                # build experiment parameters
                                experiment = {}

                                experiment['architecture'] = 'video_LRCNN_frozen'
                                experiment['pretrained_model_name'] = pretrained_model_name
                                experiment['layer_1_size'] = layer_1_size
                                experiment['layer_2_size'] = layer_2_size
                                experiment['layer_3_size'] = layer_3_size
                                experiment['dropout'] = dropout
                                experiment['pooling'] = 'max' # outperforms avg across all parameters
                                experiment['sequence_model'] = sequence_model
                                experiment['sequence_model_layers'] = sequence_model_layers

                                # add to list of experiments
                                experiments.append(experiment)

In [28]:
########################
### convert to dataframe
########################

experiments = pd.DataFrame(experiments)
experiments['model_id'] = experiments.index

In [30]:
experiments['WORKER'] = experiments['model_id'].apply(lambda x: x % WORKERCOUNT)

TypeError: unsupported operand type(s) for %: 'int' and 'NoneType'

In [ ]:
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv')

In [ ]:
pretrained_model_names_bucketed

In [ ]:
experiments.shape

In [ ]:
ee = list(range(0,101))

In [ ]:
model_id = 46

In [ ]:
###################
### Run experiments
###################

for row in experiments.values:
    
    # get experiment params from dataframe row
    experiment = dict(zip(experiments.columns, row))
    print(experiment)
    
    # only run experiment if not already run
    if not os.path.exists(pwd + 'models/' + str(model_id) + '/results.json'):

        logging.info("Begin experiment for model_id={}".format(experiment['model_id']))

        architecture = Architecture(model_id = experiment['model_id'], 
                                    architecture = experiment['architecture'], 
                                    sequence_length = experiment['sequence_length'], 
                                    pretrained_model_name = experiment['pretrained_model_name'],
                                    pooling = experiment['pooling'],
                                    sequence_model = experiment['sequence_model'],
                                    sequence_model_layers = experiment['sequence_model_layers'],
                                    layer_1_size = experiment['layer_1_size'],
                                    layer_2_size = experiment['layer_2_size'],
                                    layer_3_size = experiment['layer_3_size'],
                                    dropout = experiment['dropout'],
                                    verbose=True)

        architecture.train_model()

## batch 2

In [ ]:
#######################
### image_MLP_trainable
#######################

architecture = 'video_LRCNN_trainable'

In [ ]:
#########################
### video_LRCNN_trainable
#########################

architecture = 'video_LRCNN_trainable'

## batch 3

In [ ]:
# TODO - custom weights MLP loaded into trainable and LRCNN

## batch 4

In [ ]:
#######
### C3D
#######

architecture = 'C3D'

############
### C3Dsmall
############

architecture = 'C3Dsmall' 

# Run experiments

In [ ]:
# architecture = Architecture(model_id = 1221, 
#                             architecture = 'image_MLP_trainable',
#                             sequence_length = 1, 
#                             pretrained_model_name = "vgg16", 
#                             pooling = 'max', 
#                             layer_1_size=32,
#                             layer_2_size=0, 
#                             layer_3_size=0,
#                             dropout=0.2,
#                             verbose=True)

In [ ]:
architecture = Architecture(model_id = 55, 
                            architecture = 'image_MLP_frozen',
                            sequence_length = 1, 
                            pretrained_model_name = "vgg16", 
                            pooling = 'max', 
                            layer_1_size=256,
                            layer_2_size=128, 
                            layer_3_size=0,
                            dropout=0.2,
                            verbose=True)

In [ ]:
architecture.train_model()

# Debug

## image_MLP_trainable

In [ ]:
pretrained_model_name = "vgg16"
pooling="max"
sequence_length = 2

layer_1_size = 128
layer_2_size = 64
layer_3_size = 32
dropout=0.20

data = Data(sequence_length = 1, 
            return_CNN_features = False, 
            pretrained_model_name=pretrained_modesl_name,
            pooling = pooling,
            return_generator=True,
            batch_size=32)

num_classes = data.num_classes 
frame_size = data.frame_size
num_features = pretrained_model_len_features[pretrained_model_name]

In [ ]:
architecture = Architecture(model_id = 1, 
                            architecture = 'image_MLP_trainable',
                            sequence_length = 1, 
                            num_classes = num_classes, 
                            pretrained_model_name = pretrained_model_name, 
                            pooling = 'max', 
                            layer_1_size=128,
                            layer_2_size=0, 
                            layer_3_size=0,
                            dropout=0.2)

In [ ]:
fit(model_id=1337, model = architecture.model, data = data)

## image_MLP_frozen

In [ ]:
pretrained_model_name = "vgg16"
pooling="max"
layer_1_size = 128
layer_2_size = 64
layer_3_size = 32
dropout=0.20

In [ ]:
data = Data(sequence_length = 1, 
            return_CNN_features = True, 
            pretrained_model_name=pretrained_model_name,
            pooling = pooling)

In [ ]:
num_classes = data.num_classes 
frame_size = data.frame_size
num_features = pretrained_model_len_features[pretrained_model_name]

In [ ]:
architecture = Architecture(model_id = 1, 
                            architecture = 'image_MLP_frozen',
                            sequence_length = 1, 
                            num_classes = num_classes, 
                            pretrained_model_name = pretrained_model_name, 
                            pooling = 'max', 
                            layer_1_size=128,
                            layer_2_size=0, 
                            layer_3_size=0,
                            dropout=0.2)

In [ ]:
# train model
fit_history = fit(model_id = 1, model = architecture.model, data = data, learning_rate = 0.001, epochs = 10)

## video_MLP_concat

In [ ]:
pretrained_model_name = "vgg16"
pooling="max"
sequence_length = 3
layer_1_size = 128
layer_2_size = 64
layer_3_size = 32
dropout=0.20

In [ ]:
data = Data(sequence_length = sequence_length, 
            return_CNN_features = True, 
            pretrained_model_name=pretrained_model_name,
            pooling = pooling)

In [ ]:
num_classes = data.num_classes 
frame_size = data.frame_size
num_features = pretrained_model_len_features[pretrained_model_name]

In [ ]:
architecture = Architecture(model_id = 1, 
                            architecture = 'video_MLP_concat',
                            sequence_length = 3, 
                            num_classes = num_classes, 
                            pretrained_model_name = pretrained_model_name, 
                            pooling = 'max', 
                            layer_1_size=128,
                            layer_2_size=0, 
                            layer_3_size=0,
                            dropout=0.2)

In [ ]:
# train model
fit_history = fit(model_id = 1, model = architecture.model, data = data, learning_rate = 0.001, epochs = 10)

## video_LRCNN_frozen

In [ ]:
pretrained_model_name = "vgg16"
pooling="max"
sequence_length = 
layer_1_size = 128
layer_2_size = 64
layer_3_size = 32
dropout=0.20

In [ ]:
data = Data(sequence_length = sequence_length, 
            return_CNN_features = True, 
            pretrained_model_name=pretrained_model_name,
            pooling = pooling)

In [ ]:
architecture = Architecture(model_id=1,
                            architecture="video_LRCNN_frozen", 
                            sequence_model = 'LSTM',
                            sequence_model_layers = 1,
                            sequence_length = sequence_length,
                            num_classes = data.num_classes, 
                            frame_size = data.frame_size, 
                            pretrained_model_name='vgg16', 
                            pooling='max',
                            layer_1_size=64, 
                            layer_2_size=32, 
                            layer_3_size=8, 
                            dropout=0.2,
                            convolution_kernel_size=3)

In [ ]:
# train model
fit_history = fit(model_id=1, model=architecture.model, data=data, learning_rate = 0.001, epochs = 50)

## video_LRCNN_trainable

In [ ]:
sequence_length=5

In [ ]:
data = Data(sequence_length = sequence_length, 
            return_CNN_features = False, 
            pretrained_model_name="vgg16",
            pooling = "max",
            batch_size=32,
            return_generator=True)

In [ ]:
architecture = Architecture(model_id = 4,
                            architecture="video_LRCNN_trainable", 
                            sequence_model = 'LSTM',
                            sequence_model_layers = 1,
                            sequence_length = sequence_length,
                            num_classes = data.num_classes, 
                            frame_size = data.frame_size, 
                            pretrained_model_name='vgg16', 
                            pooling='max',
                            layer_1_size=64, 
                            layer_2_size=32, 
                            layer_3_size=8, 
                            dropout=0.2,
                            convolution_kernel_size=3)

In [ ]:
# train model
fit_history = fit(model_id = 1, model = architecture.model, data = data, learning_rate = 0.001, epochs = 50)

## C3D

In [ ]:
architecture = Architecture(model_id = 4,
                            architecture="C3D", 
                            sequence_length = 16,
                            num_classes = data.num_classes, 
                            frame_size = data.frame_size)

In [ ]:
# train model
fit_history = fit(model_id = 122, model = architecture.model, data = data, learning_rate = 0.001, epochs = 10)

## C3Dsmall

In [ ]:
data = Data(sequence_length = 16, 
            return_CNN_features = False, 
            frame_size = (112,112),
            return_generator=True,
            batch_size=32,
            verbose=False)

In [ ]:
architecture = Architecture(model_id = 4,
                            architecture="C3Dsmall", 
                            sequence_length = 16,
                            num_classes = data.num_classes, 
                            frame_size = data.frame_size)

In [ ]:
# train model
fit_history = fit(model_id = 122, model = model, data = data, learning_rate = 0.001, epochs = 10)

# analyze results

In [ ]:
path_models = pwd + 'models/'

results = []

for folder, subs, files in os.walk(path_models):
    for filename in files:
        if 'results.json' in filename:
            with open(os.path.abspath(os.path.join(folder, filename))) as f:
                data = json.load(f)
            results.append(data)

results = pd.DataFrame(results)        
results.sort_values("fit_val_acc", inplace=True, ascending=False)

In [ ]:
results.head(10)